# About: マシンのセキュリティ設定

研究クラウド/AICで確保したマシンに対して、基本的なセキュリティ設定を実施する。

**このNotebookをここに掲載しているのは、運用プロセスの一環としてセキュリティに関するタスクが明示的に書かれかつその内容を運用者が共有すべきであるということを主張のためです。**

**そのため、こんな感じで書けるというナイーブな参考です。**

## Operation Note

*ここに経緯を記述*

# Notebookと設定対象とのBinding

セキュリティ設定対象を確認する。
Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'test'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.105 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# セキュリティ設定タスクの実行

セキュリティ設定はAnsible Roleとして定義している。(NII所内からのみ/登録ユーザのみアクセス可能なRepositoryに配置)

http://xxxxxx/nii-security-setting

## Ansible用ファイルの準備

運用上必要な標準的セキュリティ設定が整備されているという前提です。ここでは、NIIクラウド担当での管理作業手順を例示的に示しています。

**具体的な設定の内容は参考です。**

In [3]:
import tempfile
temp_dir = tempfile.mkdtemp()
temp_dir

'/tmp/tmpz1R2lo'

In [4]:
!mkdir {temp_dir}/roles
!git clone ssh://git@xxxxxx/nii-security-setting.git \
           {temp_dir}/roles/nii-security-setting
!tree {temp_dir}

Cloning into '/tmp/tmpz1R2lo/roles/nii-security-setting'...
remote: Counting objects: 27, done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 27 (delta 2), reused 0 (delta 0)
Receiving objects: 100% (27/27), 5.04 KiB | 0 bytes/s, done.
Resolving deltas: 100% (2/2), done.
Checking connectivity... done.
/tmp/tmpz1R2lo
└── roles
    └── nii-security-setting
        ├── files
        │   └── yazawa.keys
        ├── handlers
        │   ├── main.yml
        │   └── sshd.yml
        ├── tasks
        │   ├── login-users.yml
        │   ├── main.yml
        │   ├── sshd.yml
        │   └── sudoers.yml
        └── vars
            ├── CentOS.yml
            └── Ubuntu.yml

6 directories, 15 files


Gitからcloneしたroleをどの対象に適用するかはPlaybookにより記述する。

In [5]:
import os
with open(os.path.join(temp_dir, 'site.yml'), 'w') as f:
    f.write('''- hosts: {}
  become: yes
  roles:
    - nii-security-setting'''.format(target_group))
!cat {temp_dir}/site.yml

- hosts: test
  become: yes
  roles:
    - nii-security-setting

In [6]:
!mkdir {temp_dir}/group_vars

セキュリティ設定は group_vars により与える。**環境に合わせて適切に設定すること！**

In [7]:
%%writefile {temp_dir}/group_vars/{target_group}
login_users:
  - { name: yazawa, comment: "Satoshi Yazawa", key: "yazawa.keys" }
  
sshd_allow_hosts:
  - "XXX.XXX.XXX.0/24"
  - "XXX.XXX.XXX.0/24"
  - "XXX.XXX.XXX.0/23"
  - "XXX.XXX.XXX.0/24"
  - "XXX.XXX.XXX.106"
  - "XXX.XXX.XXX.110"
  - "XXX.XXX.XXX.192/26"

Writing /tmp/tmpz1R2lo/group_vars/test


## Ansibleの実行

まずはCheck modeで動作確認してみる。

In [8]:
!ansible-playbook -CDv {temp_dir}/site.yml

Using /etc/ansible/ansible.cfg as config file

PLAY [test] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [nii-security-setting : Gather OS Specific Variables] *********************
ok: [XXX.XXX.XXX.105] => (item=/tmp/tmpz1R2lo/roles/nii-security-setting/vars/CentOS.yml) => {"ansible_facts": {"sshd_service": "sshd"}, "item": "/tmp/tmpz1R2lo/roles/nii-security-setting/vars/CentOS.yml"}

TASK [nii-security-setting : include] ******************************************
included: /tmp/tmpz1R2lo/roles/nii-security-setting/tasks/sudoers.yml for XXX.XXX.XXX.105

TASK [nii-security-setting : Prepare /etc/sudoers.d] ***************************
changed: [XXX.XXX.XXX.105] => {"changed": true, "gid": 0, "group": "root", "mode": "0750", "owner": "root", "path": "/etc/sudoers.d", "size": 4096, "state": "directory", "uid": 0}
--- before
+++ after
@@ -1,4 +1,4 @@
 {
-    "mode": "075

問題なさそう。適用する。

In [9]:
!ansible-playbook {temp_dir}/site.yml


PLAY [test] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.105]

TASK [nii-security-setting : Gather OS Specific Variables] *********************
ok: [XXX.XXX.XXX.105] => (item=/tmp/tmpz1R2lo/roles/nii-security-setting/vars/CentOS.yml)

TASK [nii-security-setting : include] ******************************************
included: /tmp/tmpz1R2lo/roles/nii-security-setting/tasks/sudoers.yml for XXX.XXX.XXX.105

TASK [nii-security-setting : Prepare /etc/sudoers.d] ***************************
changed: [XXX.XXX.XXX.105]

TASK [nii-security-setting : Add #includedir to /etc/sudoers] ******************
ok: [XXX.XXX.XXX.105]

TASK [nii-security-setting : include] ******************************************
included: /tmp/tmpz1R2lo/roles/nii-security-setting/tasks/login-users.yml for XXX.XXX.XXX.105

TASK [nii-security-setting : Add user] *****************************************
changed: [

SSHでアクセスできることを確認し、クローズとする。

# 後始末

一時ディレクトリを削除する。

In [10]:
!rm -fr {temp_dir}